In [1]:
import json
from typing import List, Dict, Any
from tqdm import tqdm

from llm_client import get_llama
from llm_prompts import VACANCY_COMPETENCIES_PROMPT

import gc as py_gc
import os
from datasets import load_dataset
from llm_client import MODEL_NAME

## Обработка вакансий моделью с или без QLoRA

In [6]:
# src/analyze_vacancies_llm.py
from llm_client import reset_llama
BATCH_SIZE = 7
MAX_NEW_TOKENS = 128
TRACE_PATH = "vac_test.txt"

def load_vacancies(path: str) -> List[Dict[str, Any]]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def _build_prompt(vac: Dict[str, Any]) -> str:
    industry = vac.get("industry")
    title = vac.get("title")
    description = vac.get("description") or ""
    skills_extracted = vac.get("skills_extracted") or []

    return VACANCY_COMPETENCIES_PROMPT.format(
        industry=industry,
        title=title,
        description=description[:4000],  # safety-ограничение
        skills_extracted=skills_extracted,
    )

def _write_trace(f, meta: Dict[str, Any], prompt: str, answer: str) -> None:
    f.write("\n" + "=" * 120 + "\n")
    f.write(f"vacancy_id: {meta.get('vacancy_id')}\n")
    f.write(f"industry: {meta.get('industry')}\n")
    f.write(f"title: {meta.get('title')}\n")
    f.write("-" * 120 + "\n")
    f.write("PROMPT:\n")
    f.write(prompt.rstrip() + "\n")
    f.write("-" * 120 + "\n")
    f.write("MODEL_OUTPUT:\n")
    f.write((answer or "").rstrip() + "\n")
    f.write("=" * 120 + "\n")

def use_qlora(use: bool = True,
              adapter: str = r"QLoRA/vac_qlora_adapter/checkpoint-200"):
    ADAPTER_DIR = None
    if use:
        ADAPTER_DIR = adapter
    return ADAPTER_DIR

reset_llama()
print("Память очищена")
def analyze_vacancies(vacancies_path: str):
    adapter = r"QLoRA/vac_qlora_adapter/checkpoint-200"  # путь до QLoRA
    ADAPTER_DIR = use_qlora(False)
    if ADAPTER_DIR:
        print("adapter_dir:", ADAPTER_DIR)
        print("exists:", os.path.exists(ADAPTER_DIR))
        print("adapter_config exists:", os.path.exists(os.path.join(ADAPTER_DIR, "adapter_config.json")))
    llama = get_llama(adapter_dir=ADAPTER_DIR)
    try:
        vacancies = load_vacancies(vacancies_path)

        batch_prompts: List[str] = []
        batch_meta: List[Dict[str, Any]] = []

        with open(TRACE_PATH, "w", encoding="utf-8") as trace_f:
            trace_f.write("# vac_test.txt — prompts + raw model outputs (one block per vacancy)\n")

            def process_batch():
                nonlocal batch_prompts, batch_meta
                if not batch_prompts:
                    return

                if hasattr(llama, "ask_batch"):
                    raw_answers = llama.ask_batch(
                        batch_prompts,
                        max_new_tokens=MAX_NEW_TOKENS,
                        batch_size=BATCH_SIZE,
                    )
                else:
                    raw_answers = [
                        llama.ask_one(p, max_new_tokens=MAX_NEW_TOKENS)
                        for p in batch_prompts
                    ]

                for meta, prompt, raw in zip(batch_meta, batch_prompts, raw_answers):
                    _write_trace(trace_f, meta, prompt, raw)

                batch_prompts = []
                batch_meta = []

            for vac in tqdm(vacancies, desc="LLM: vacancies (trace to vac_test.txt)"):
                prompt = _build_prompt(vac)

                batch_prompts.append(prompt)
                batch_meta.append({
                    "vacancy_id": vac.get("id"),
                    "industry": vac.get("industry"),
                    "title": vac.get("title"),
                })

                if len(batch_prompts) >= BATCH_SIZE:
                    process_batch()

            process_batch()

        print(f"[OK] Traces saved to {TRACE_PATH}")

    finally:
        try:
            llama.close()
        except Exception:
            pass
        del llama  # важно в ноутбуке
    
        from llm_client import reset_llama
        reset_llama()
    
        import gc, torch
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()


if __name__ == "__main__":
    analyze_vacancies("data/processed/vacancies_processed.json")


Память очищена
[LLM] Проверяем и докачиваем модель (snapshot_download)...


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

[LLM] Загружаем модель из /home/snaw/.cache/huggingface/hub/models--meta-llama--Llama-3.1-8B-Instruct/snapshots/0e9e39f249a16976918f6564b8830bc894c89659


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


[LLM] LlamaClient готов к работе.


LLM: vacancies (trace to vac_test.txt): 100%|██████████| 311/311 [08:47<00:00,  1.70s/it]
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


[OK] Traces saved to vac_test.txt


## Очистка памяти

In [7]:
# ===== GPU / LLM MEMORY CLEANUP =====
"""
эффективнее будет
nvidia-smi | grep python
pkill -9 python
но это убьет вообще все py процессы
"""
def cleanup_llm_memory():
    try:
        import llm_client

        # если llama_client существует — закрываем модель
        llama = getattr(llm_client, "llama_client", None)
        if llama is not None:
            try:
                llama.close()
                print("[cleanup] llama.close() called")
            except Exception as e:
                print("[cleanup] llama.close() failed:", e)

        # сбрасываем singleton
        if hasattr(llm_client, "reset_llama"):
            llm_client.reset_llama()
            print("[cleanup] llama_client reset")

    except Exception as e:
        print("[cleanup] llm_client not found or already cleaned:", e)

    for name in ["model", "base", "pipe", "trainer", "optimizer", "tokenizer"]:
        if name in globals():
            del globals()[name]
    
    import torch
    py_gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    
    print("allocated:", torch.cuda.memory_allocated()/1024**2, "MB")
    print("reserved :", torch.cuda.memory_reserved()/1024**2, "MB")

    print("[cleanup] DONE")

cleanup_llm_memory()


[cleanup] llama_client reset
allocated: 0.0009765625 MB
reserved : 2.0 MB
[cleanup] DONE


In [2]:
## Конфиг и загрузка датасета JSONL

In [2]:
DATA_PATH = "vac_qloRA_train_v2.jsonl"   
OUT_DIR = "QLoRA/vac_qlora_adapter"

# Загружаем JSONL как dataset
ds = load_dataset("json", data_files=DATA_PATH, split="train")

# Быстрый sanity-check
print(ds)
print(ds[0].keys())
print("пример prompt:", ds[0]["prompt"][:200].replace("\n"," ") + " ...")
print("пример response:", ds[0]["response"])


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['prompt', 'response'],
    num_rows: 311
})
dict_keys(['prompt', 'response'])
пример prompt: Проанализируй текст вакансии и выдели ТОЛЬКО профессиональные компетенции из текста вакансии. Возможными компетенциями могут быть: языки программирования, фреймворки, библиотеки, базы данных и методы  ...
пример response: ["разработка на 1С", "сопровождение 1С", "написание ТЗ", "интеграции", "SQL", "тестирование доработок", "консультация пользователей"]


## Сплит train/val и сборка текста для SFT

In [3]:
ds = ds.train_test_split(test_size=0.05, seed=42)
train_ds, eval_ds = ds["train"], ds["test"]

def format_example(ex):

    text = ex["prompt"].strip() + "\n\n" + ex["response"].strip()
    return {"text": text}

train_ds = train_ds.map(format_example, remove_columns=train_ds.column_names)
eval_ds  = eval_ds.map(format_example, remove_columns=eval_ds.column_names)

print(train_ds[0]["text"][:500], "...\n")


Map:   0%|          | 0/295 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Проанализируй текст вакансии и выдели ТОЛЬКО профессиональные компетенции из текста вакансии.
Возможными компетенциями могут быть: языки программирования, фреймворки, библиотеки, базы данных и методы работы с ними, разработка чат-ботов, 
направления в индустриях, например построение ML-моделей, NLP, LLM, а также A/B-тестирование, геймдизайн, Power BI, разработка онлайн-курсов.

Данные вакансии:
Отрасль: EdTech
Название: Оператор онлайн-чата
Описание: Работа в офисе !!!!!
Обязанности
* Ответы на  ...



In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer

### Конфиг

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=True,
)

## Загрузка модели (4-bit QLoRA) 

In [6]:
# pad_token нужен для батчинга
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

model = prepare_model_for_kbit_training(model)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Обучение (SFTTrainer) и сохранение адаптера

In [8]:
import trl # разные версии работают абсолютно по-разному
print(trl.__version__)

0.26.1


In [9]:
from trl import SFTTrainer, SFTConfig

# LoRA config (классический QLoRA сетап)
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

args = SFTConfig(
    output_dir=OUT_DIR,
    max_length=2048, # входящих токенов может быть до 1800
    packing=False,
    num_train_epochs=8,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=3,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    save_total_limit=5,
    optim="paged_adamw_8bit",
    report_to="none",
    logging_strategy="steps",   
    logging_first_step=True,    
    completion_only_loss=True,
    bf16=False,
    fp16=True,
    max_grad_norm=0.0,
)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    processing_class=tokenizer,
)

# КЛЮЧЕВОЕ: отключаем GradScaler
if hasattr(trainer, "accelerator") and hasattr(trainer.accelerator, "scaler"):
    trainer.accelerator.scaler = None
    print("[patch] GradScaler disabled")

trainer.train()

trainer.model.save_pretrained(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

print(f"[OK] Adapter saved to: {OUT_DIR}")


trainable params: 83,886,080 || all params: 8,114,147,328 || trainable%: 1.0338


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.


[patch] GradScaler disabled


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,1.018700,0.962100,1.053691,130373.000000,0.782306
100,0.726700,0.746448,0.827214,250001.000000,0.829311
150,0.496900,0.710210,0.657830,373966.000000,0.841228
200,0.427600,0.623209,0.592642,501593.000000,0.858397
250,0.386600,0.641123,0.535451,623666.000000,0.859420


KeyboardInterrupt: 

## Быстрый тест

### выгрузка модели

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
from llm_client import MODEL_NAME

#ADAPTER_DIR = OUT_DIR  # или r"RH-AI\RH AI\src\QLoRA\vac_qlora_adapter"
ADAPTER_DIR = r"QLoRA/vac_qlora_adapter/checkpoint-200"
# 4-bit config (как в обучении)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# базовая модель целиком на GPU:0 (без CPU/disk dispatch)
base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": 0},
    torch_dtype=torch.float16,
)

# подключаем QLoRA адаптер
model = PeftModel.from_pretrained(base, ADAPTER_DIR)
model.eval()

# генератор
gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map={"": 0},
    return_full_text=False,
    pad_token_id=tokenizer.pad_token_id,
)

print("[OK] base+LoRA loaded, gen_pipe is ready")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


[OK] base+LoRA loaded, gen_pipe is ready


### Сам тест

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

TEST_PROMPT = """Проанализируй текст вакансии и выдели ТОЛЬКО профессиональные компетенции из текста вакансии.
Возможными компетенциями могут быть: языки программирования, фреймворки, библиотеки, базы данных и методы работы с ними, разработка чат-ботов, 
направления в индустриях, например построение ML-моделей, NLP, LLM, а также A/B-тестирование, геймдизайн, Power BI, разработка онлайн-курсов.

Данные вакансии:
Отрасль: 1CDevelopment
Название: IT-специалист / Инженер 1 линии технической поддержки
Описание: Описание вакансии
Компания "РОСТ" - системный интегратор в области информационных технологий и Партнёр ведущих российских разработчиков Программного обеспечения, таких как: СБИС; 1С Bitrix; LiteBox и другие.
Обязанности:
• Принимать «горячие» обращения от клиентов
• преимущественно звонки, а также мессенджеры;
• правильно оформлять и заносить заявки в Help Desk;
• удаленно оперативно устранять сбои в работе программно-аппаратного комплекса клиента
• (система для автоматизации предприятий.);
• правильно настраивать сеть, торговое оборудование и компьютерную технику;
• качественно консультировать и обучать клиентов;
• быть готовым работать с конфликтными клиентами;
• уметь работать в команде.
• выезд, если нет возможности решить проблему удаленно;
• ведение и поддержание в актуальном состоянии документации;
Требования:
• опыт работы системным администратором или инженером технической поддержки (helpdesk/servicedesk) приветствуется;
• опыт решения пользовательских задач (проблемы с офисным ПО, оргтехникой, работа с общими папками, сетевая печать, ЭЦП и т. п.) приветствуется;
• опыт работы с антивирусным ПО (в т. ч. с централизованной консолью управления (в основном мы используем Kaspersky, Dr. Web);
• плюсом будет наличие сертификатов (в области
• ИТ
• плюсом будет опыт администрирования 1С;
• плюсом будет опыт работы в Битрикс24;
...
Формат ответа:
["компетенция1", "компетенция2", ...]
Ответ:"""

test2 = """Проанализируй текст вакансии и выдели ТОЛЬКО профессиональные компетенции из текста вакансии.
Возможными компетенциями могут быть: языки программирования, фреймворки, библиотеки, базы данных и методы работы с ними, разработка чат-ботов, 
направления в индустриях, например построение ML-моделей, NLP, LLM, а также A/B-тестирование, геймдизайн, Power BI, разработка онлайн-курсов.

Данные вакансии:
Отрасль: ChatBots
Название: Продавец-кассир (ТЦ Макси Сопот, Приморская)
Описание: Familia - основоположник и лидер российского off-price ритейла - благодаря своей товарной и ценовой политике, широте и частоте обновление ассортимента. В магазинах сети представлен широкий ассортимент мужской, женской и детской одежды и обуви, аксессуары, игрушки, весь спектр товаров для дома и декора, товары для домашних питомцев и многое другое по максимально выгодным ценам.
Уважаемые кандидаты, мы ждём Вас на собеседование в магазине Familia со вторника по субботу с 10:00 до 19:00 БЕЗ ПРЕДВАРИТЕЛЬНОЙ ЗАПИСИ.
По дополнительным вопросам, пожалуйста, звоните по указанному номеру телефона или пишите в чат, WhatsApp.
Мы предлагаем:
• Оформление по ТК РФ с первого дня работы, оплачиваемый больничный лист, отпуск.
• Официальная белая заработная плата, выплачивается два раза в месяц.
• Стабильный оклад + ежемесячная премия 15% по итогам работы магазина.
• Отсутствие личных продаж!
• График работы 2/2.
• Предоставление форменной одежды.
• Корпоративное обучение.
• Перспектива карьерного роста до Заместителя управляющего магазином, Управляющего магазином.
• Бонус по акции "Приведи друга" до 5000 руб.
• Программа лояльности для сотрудников по системе кэшбек баллов.
• Новогодние подарки для детей сотрудников.
• Дополнительные конкурсные программы с денежным вознаграждением.
• Бонусные программы для сотрудников (скидки от партнеров: красота, спорт, развлечения, электроника, туризм и многое другое).
Основные задачи:
• Работа за кассовым терминалом, обслуживание покупателей на кассе.
• Поддержание порядка, соблюдение стандартов визуального мерчендайзинга и выкладка товара в прикассовой зоне
• Участие в инвентаризации (1 раз в год).
Для нас важно:
• Активность и готовность обучаться.
• Ответственность, внимательность, доброжелательность.
• Имеете опыт работы – отлично! Без опыта – обучим!
Приветствуется опыт работы на вакансиях: продавец, без опыта, менеджер по продажам, консультант, кассир, менеджер по работе с клиентами, начинающий специалист, ртз, работник торгового зала, кассир с ежедневными выплатами в компаниях:
Gloria Jeans, Глория Джинс, Sela, Zarina, Зарина, Детский мир, Дочки-сыночки, Defacto, LC Waikiki, Sinsay, Sin, Синсей, Reserved, H&M, Cropp, House, Modis, Модис, Decatlon, Декатлон, MAAG, ECRU, DUB, VILET, Новая мода, Zara, Gloria-Jeans, GJ, Зара, Incity, Инсити, Inditex, Bershka, Massimo Dutti, Uterque, Pull&Bear, Stradivarius, Oysho, Lefties, Ойшо, Бершка, Пулл энд Беар, Массимо Дутти, Лефтиес, Kari, Кари, Intimissimi, Calzedonia, Кальзедония, Incanto, Calvin Klein, Collins, Коллинс, ТВОЕ, Levi’s, Mango, 12Storeez, Подружка, 5 карманов, Koton, Котон, Love Republic, New Yorker, Terranova, Терранова, Finn Flare, Фин Флэр, Zolla, Золла, Oodji, Оджи, Adidas, МВидео, Леруа Мерлен, OBI, SuperStep, Триал-Спорт, ZENDEN, KIABI, Киаби, RESPECT, TAMARIS, TERVOLINA, Хоум Маркет, Терволина, Nike, PUMA, Columbia, Modis, Demix, Mango, МВидео, Terranova, Эльдорадо, ИКЕА, Zolla, Oodj, Love Republic, Reebok, Связной, МТС, Мегафон, Билайн, Adidas, New Balance, Decathlon, Puma, Reebok, New Balance, Fun Day, Фандэй, Фандей, Sportmaster, Befre, Мохито, Mohito, Benetton, Intimissimi, Интимисими, Calzedonia, Калзедония, Incanto, Инканто, Incity, Инсити, Uniqlo, Юникло, Подружка, SuperStep, Zenden, Kiabi, Киаби, Respect, Rendez-vous, Рандеву, Терволина, МВидео, Эльдорадо, ИКЕА, Леруа Мерлен, OBI, Hoff, Хоф, Твой Дом, Галамарт, Хоум Маркет, DNS, Связной, МТС, Мегафон, Билайн, Вкусно и точка, Макдоналдс, Золотое яблоко, Idol, Yandex, KFC, Бургер Кинг, Burger King, Ростикс, Иль де боте, Fix price, Off price, Сбербанк, Тинькофф, Tinkoff, СДЭК, OZON, WILDBERRIES, Остин, Озон, Валдберрис, Kidzania, Кидзания, Совкомбанк, Альфа-Банк, Додо Пицца, Melon Fashion Group, Apple, Samsung, IL Патио, Шоколадница, Кофе Хаус, Cofix, Stars coffee, Surf Coffee, Gate31, 12 Cторис, Bell you, Джинсовая Симфония, Jeans Symphony, Ривгош, ИвРоше, Tous, Yves Rocher, One&Double, Double coffee, Star Hit, Nice Price, Bubble tea, Братья Караваевы, Гурманика, Азбука Вкуса, Вкусви

Верни только JSON-массив строк. Компетенций(строк) в массиве должно быть не менее одной, но НЕ БОЛЕЕ 7. Компетенций должны быть выделены строго из текста вакансии.
Не добавляй в массив строк компетенции, не относящиеся к вакансии.
Формат ответа:
["компетенция1", "компетенция2", ...]
Ответ:"""
prompts = [
    ("1c test", TEST_PROMPT),
    ("Продавец test", test2),
]

def count_tokens(text: str) -> int:
    t = tokenizer(text, return_tensors=None, add_special_tokens=False)
    return len(t["input_ids"])

print("Token counts:")
for name, p in prompts:
    print(f"{name}: {count_tokens(p)}")

print("\n=== QLoRA QUICK TEST ===")
with torch.inference_mode():
    for name, p in prompts:
        out = gen_pipe(
            p,
            max_new_tokens=128,
            min_new_tokens=2,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
        )[0]["generated_text"]

        print(f"\n--- {name} ---")
        print(out)

Token counts:
1c test: 617
Продавец test: 1817

=== QLoRA QUICK TEST ===

--- 1c test ---
 ["телефонная поддержка", "работа с клиентами", "решение технических задач", "консультирование", "ведение документации", "работа с ПО", "анализ проблем", "поиск решений", "тестирование"]

--- Продавец test ---
 ["работа с клиентами", "ведение продаж", "ведение кассовой операции", "обслуживание покупателей", "работа с товарным ассортиментом", "поддержание чистоты и порядка", "коммуникация с коллегами"]
